In [1]:
# Import base dependencies
import pandas as pd
import requests
import time
from datetime import datetime
import math
from config import av_api_key as api_key
# import matplotlib.pyplot as plt
# from sklearn.linear_model import LinearRegression

# Import ML dependencies
# import numpy as np
# import tensorflow as tf
# from tensorflow import keras
# from keras.models import Sequential
# from keras.layers import SimpleRNN, Flatten, TimeDistributed, LSTM

In [2]:
# Import raw data from platform export
# IRA funds
ira_funds = [{'symbol':"UNFI"}, 
             {'symbol':"MFC"}, 
             {'symbol':"EAT", 'basis':126.365}, 
             {'symbol':"EZPW", 'basis':15.46}, 
             {'symbol':"ARQT", 'basis':15.932}, 
             {'symbol':"WFC", 'basis':66.113}, 
             {'symbol':"ITRN"}, 
             {'symbol':"CRDO", 'basis':35.613}, 
             {'symbol':"PYPL"}, 
             {'symbol':"ALL"}, 
             {'symbol':"LC"}, 
             {'symbol':"QTWO"}, 
             {'symbol':"CLS", 'basis':69.342}, 
             {'symbol':"CCL", 'basis':17.999}, 
             {'symbol':"AGX", 'basis':93.089}, 
             {'symbol':"POWL", 'basis':187.000}, 
             {'symbol':"PPC", 'basis':45.45}, 
             {'symbol':"SYF"}, 
             {'symbol':"ATGE"}, 
             {'symbol':"BRK-B"}, 
             {'symbol':"SFM"}, 
             {'symbol':"SKYW", 'basis':83.177}, 
             {'symbol':"BLBD"}, 
             {'symbol':"GM"}, 
             {'symbol':"RCL"}, 
             {'symbol':"OKTA"}, 
             {'symbol':"TWLO", 'basis':100.519}, 
             {'symbol':"APP", 'basis':276.123}, 
             {'symbol':"TMUS"}, 
             {'symbol':"STRL"}, 
             {'symbol':"GRBK"}, 
             {'symbol':"UBER", 'basis':54.777}, 
             {'symbol':"CAAP"}
             ]

# Brokerage
brokerage_funds = [{'symbol':"AGX", 'basis':86.654}, 
                   {'symbol':"APP", 'basis':225.619}, 
                   {'symbol':"ARQT", 'basis':15.996}, 
                   {'symbol':"ATGE", 'basis':86.385}, 
                   {'symbol':"CCL", 'basis':20.829}, 
                   {'symbol':"CLS", 'basis':74.924}, 
                   {'symbol':"CRDO", 'basis':38.629}, 
                   {'symbol':"EAT", 'basis':135.74}, 
                   {'symbol':"EZPW", 'basis':18.187}, 
                   {'symbol':"FBTC", 'basis':75.244, 'is_etf': True}, 
                   {'symbol':"NVDA", 'basis':95.194}, 
                   {'symbol':"SKYW", 'basis':84.188}, 
                   {'symbol':"TWLO", 'basis':92.001}, 
                   {'symbol':"WFC", 'basis':72.705}
                   ]

In [3]:
def get_history(symbol, api_key, days=252):  # ~1 year default
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={symbol}&apikey={api_key}&outputsize=full&entitlement=delayed"
    try:
        response = requests.get(url).json()
        if "Time Series (Daily)" not in response:
            error_msg = response.get('Note', response.get('Information', 'Unknown error'))
            print(f"Error fetching price data for {symbol}: {error_msg}")
            print(f"Response keys: {list(response.keys())}")
            return None
        
        time_series = response["Time Series (Daily)"]
        df = pd.DataFrame.from_dict(time_series, orient="index", dtype=float)
        
        # Debug: Print available columns
        # print(f"Columns for {symbol}: {list(df.columns)}")
        
        # Rename columns dynamically
        column_map = {
            col: name for col, name in [
                ("1. open", "Open"), ("2. high", "High"), ("3. low", "Low"),
                ("4. close", "Close"), ("5. volume", "Volume"), ("6. volume", "Volume"),
                ("7. adjusted close", "Adjusted Close"), ("8. dividend amount", "Dividend")
            ] if col in df.columns
        }
        if "5. volume" not in df.columns and "6. volume" not in df.columns:
            print(f"No volume data for {symbol}")
            return None
        
        df = df.rename(columns=column_map)
        df.index = pd.to_datetime(df.index)
        df = df.sort_index().tail(days)
        return df
    except Exception as e:
        print(f"Exception fetching price data for {symbol}: {str(e)}")
        return None

def get_fundamentals(symbol, api_key, current_price):
    url = f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={api_key}&entitlement=delayed"
    try:
        response = requests.get(url).json()
        if not response or "Symbol" not in response:
            error_msg = response.get('Note', response.get('Information', 'No data'))
            print(f"Error fetching fundamentals for {symbol}: {error_msg}")
            print(f"Full response: {response}")
            return None
        
        def safe_float(value, default):
            if value in [None, 'None', '']:
                return default
            try:
                return float(value)
            except (ValueError, TypeError):
                return default
        
        pe_ratio = safe_float(response.get('PERatio'), float('inf'))
        pb_ratio = safe_float(response.get('PriceToBookRatio'), float('inf'))
        
        # Calculate EPS and Book Value
        eps = current_price / pe_ratio if pe_ratio != float('inf') and pe_ratio != 0 else 0
        book_value = current_price / pb_ratio if pb_ratio != float('inf') and pb_ratio != 0 else 0
        
        fundamentals = {
            'pe_ratio': pe_ratio,
            'pb_ratio': pb_ratio,
            'dividend_yield': safe_float(response.get('DividendYield'), 0),
            'debt_to_equity': safe_float(response.get('DebtToEquityRatio'), float('inf')),
            'eps': eps,
            'book_value': book_value
        }
        
        return fundamentals
    except Exception as e:
        print(f"Exception fetching fundamentals for {symbol}: {str(e)}")
        return None

def calculate_vwap(df, days=126):  # ~2 quarters
    if 'Volume' not in df.columns:
        print("Missing Volume column in DataFrame")
        return None
    
    vwap_analysis = df[-days:].copy()
    vwap_analysis['Cumulative_LTPV'] = (vwap_analysis['Low'] * vwap_analysis['Volume']).cumsum()
    vwap_analysis['Cumulative_HTPV'] = (vwap_analysis['High'] * vwap_analysis['Volume']).cumsum()
    vwap_analysis['Cumulative_Volume'] = vwap_analysis['Volume'].cumsum()
    vwap_analysis['Entry'] = round(vwap_analysis['Cumulative_LTPV'] / vwap_analysis['Cumulative_Volume'], 2)
    vwap_analysis['Exit'] = round(vwap_analysis['Cumulative_HTPV'] / vwap_analysis['Cumulative_Volume'], 2)
    return vwap_analysis[-1:].copy()

def build_analysis_table(ticker_symbols, api_key, margin_of_safety=0.9, vwap_days=126, graham_margin=0.9):
    portfolio = []
    
    for ticker in ticker_symbols:
        symbol = ticker['symbol']
        is_etf = ticker.get('is_etf', False)
        
        # Get price data
        raw_data = get_history(symbol, api_key)
        if raw_data is None:
            portfolio.append([symbol, ticker.get('basis', 0), None, None, None, None, None, None, None, "Error"])
            continue
        
        # Get fundamentals (skip for ETFs)
        current_price = raw_data['Close'].iloc[-1]  # Use Close for fundamental calcs
        fundamentals = None if is_etf else get_fundamentals(symbol, api_key, current_price)
        if not is_etf and fundamentals is None:
            portfolio.append([symbol, ticker.get('basis', 0), None, None, None, None, None, None, None, "Error"])
            continue
        
        # Calculate VWAP
        vwap_data = calculate_vwap(raw_data, days=vwap_days)
        if vwap_data is None:
            portfolio.append([symbol, ticker.get('basis', 0), None, None, None, None, None, None, None, "Error"])
            continue
        
        # Extract data
        basis = ticker.get('basis', 0)
        market_price = round(raw_data['Close'].iloc[-1], 2)  # Use Low for buys
        entry_price = round(vwap_data['Entry'].iloc[0], 2)
        exit_price = round(vwap_data['Exit'].iloc[0], 2)
        buy_threshold = round(entry_price * margin_of_safety, 2)  # 10% margin
        
        # Graham buy threshold (for stocks only)
        graham_buy_threshold = None
        if not is_etf:
            if fundamentals['eps'] > 0 and fundamentals['book_value'] > 0:
                # Calculate desired price where P/E × P/B = 36
                desired_price = math.sqrt(38 * fundamentals['eps'] * fundamentals['book_value'])
                graham_buy_threshold = round(desired_price * graham_margin, 2)  # 5% margin
            else:
                graham_buy_threshold = buy_threshold  # Default to VWAP threshold
        
        # Volume filter: 20% of 21-day average
        avg_volume = raw_data['Volume'][-21:].mean()
        today_volume = raw_data['Volume'].iloc[-1]
        volume_ok = today_volume >= avg_volume * 0.2
        
        # Graham's fundamental checks (for stocks only)
        graham_ok = True
        if not is_etf:
            graham_ok = (
                (fundamentals['pe_ratio'] < 19 and fundamentals['pb_ratio'] < 2.0) or
                (fundamentals['pe_ratio'] * fundamentals['pb_ratio'] < 38 and 
                 fundamentals['pe_ratio'] < 100 and fundamentals['pb_ratio'] < 10)
                 ) and fundamentals['dividend_yield'] >= 0 and fundamentals['debt_to_equity'] < 2
        
        # Decision logic
        decision = "Hold"
        if market_price <= min(buy_threshold, graham_buy_threshold or float('inf')) and volume_ok and graham_ok:
            decision = "Buy"
        elif market_price >= exit_price and volume_ok:
            decision = "Sell"
        
        # Prepare fundamentals for output
        pe_ratio = None if is_etf else fundamentals['pe_ratio']
        pb_ratio = None if is_etf else fundamentals['pb_ratio']
        dividend_yield = None if is_etf else fundamentals['dividend_yield']
        
        portfolio.append([
            symbol, basis, market_price, 
            buy_threshold, graham_buy_threshold, exit_price,
            pe_ratio, pb_ratio, dividend_yield, decision
        ])
        
        # Minimal delay for server stability (75 calls/minute = ~0.8 seconds/call)
        time.sleep(0.1)
    
    return portfolio

In [4]:
b_portfolio = build_analysis_table(brokerage_funds, api_key, margin_of_safety=0.97, vwap_days=63)
brokerage_df = pd.DataFrame(b_portfolio, 
                             columns=['ticker', 'basis', 'price', 
                                      'VWAP', 'GIIB', 'exit', 
                                      'P/E', 'P/B', 'DivYield', 'rating'])

In [5]:
brokerage_df

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
0,AGX,86.654,188.50,128.88,70.79,142.55,30.70,7.110,0.0082,Sell
1,APP,225.619,362.34,276.73,17.03,315.52,65.40,213.080,0.0000,Sell
2,ARQT,15.996,14.11,13.41,13.41,15.14,inf,11.790,0.0000,Hold
3,ATGE,86.385,134.74,100.90,85.24,109.38,22.76,3.379,0.0000,Sell
4,CCL,20.829,23.39,19.01,18.14,20.71,15.09,3.393,0.0000,Sell
5,CLS,74.924,113.25,84.73,39.24,94.67,31.11,8.240,0.0000,Sell
6,CRDO,38.629,60.13,43.87,1.83,49.62,2004.33,16.520,0.0000,Sell
7,EAT,135.740,151.61,136.51,36.64,149.73,20.66,25.510,0.0000,Sell
8,EZPW,18.187,13.96,14.45,22.82,15.49,11.73,0.982,0.0000,Hold
9,FBTC,75.244,90.88,72.78,NaN,78.07,NaN,NaN,NaN,Sell


In [6]:
r_portfolio = build_analysis_table(ira_funds, api_key, margin_of_safety=0.95, vwap_days=189)
retirement_df = pd.DataFrame(r_portfolio, 
                             columns=['ticker', 'basis', 'price', 
                                      'VWAP', 'GIIB', 'exit', 
                                      'P/E', 'P/B', 'DivYield', 'rating'])

In [7]:
retirement_df[retirement_df['basis']>0]

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
2,EAT,126.365,151.61,116.90,36.64,129.13,20.66,25.510,0.0000,Sell
3,EZPW,15.460,13.96,12.91,22.82,14.09,11.73,0.982,0.0000,Hold
4,ARQT,15.932,14.11,11.33,11.33,12.99,inf,11.790,0.0000,Sell
5,WFC,66.113,76.18,63.57,93.21,68.88,13.58,1.514,0.0212,Sell
7,CRDO,35.613,60.13,47.39,1.83,54.44,2004.33,16.520,0.0000,Sell
12,CLS,69.342,113.25,81.98,39.24,93.01,31.11,8.240,0.0000,Sell
13,CCL,17.999,23.39,19.70,18.14,21.64,15.09,3.393,0.0000,Sell
14,AGX,93.089,188.50,124.54,70.79,139.98,30.70,7.110,0.0082,Sell
15,POWL,187.000,183.17,203.69,144.52,228.97,12.49,3.959,0.0060,Hold
16,PPC,45.450,48.64,45.01,46.73,49.21,9.32,3.578,0.0000,Hold


In [8]:
retirement_df[retirement_df['basis']==0]

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
0,UNFI,0.0,31.13,22.50,22.50,25.06,inf,1.139,0.0000,Sell
1,MFC,0.0,32.00,28.07,33.32,30.24,16.84,1.686,0.0401,Sell
6,ITRN,0.0,37.82,31.98,27.86,35.43,14.01,4.050,0.0530,Sell
8,PYPL,0.0,72.18,70.87,53.41,76.96,16.22,3.466,0.0000,Hold
9,ALL,0.0,209.06,181.49,186.41,195.98,14.28,2.711,0.0195,Sell
10,LC,0.0,10.64,12.25,12.72,13.62,24.18,0.890,0.0000,Hold
11,QTWO,0.0,91.01,81.32,81.32,89.05,inf,10.310,0.0000,Sell
17,SYF,0.0,60.80,53.18,95.20,58.04,8.33,1.507,0.0197,Sell
18,ATGE,0.0,134.74,88.01,85.24,96.47,22.76,3.379,0.0000,Sell
19,BRK-B,0.0,514.31,454.05,591.69,487.71,13.72,1.695,0.0000,Sell


In [9]:
# Research new investments
if 1==1:
    custom_watchlist = [{'symbol':'MSFT'}, 
                        {'symbol':'AAPL'}, 
                        {'symbol':'NVDA'}, 
                        {'symbol':'AMZN'}, 
                        {'symbol':'GOOG'}, 
                        {'symbol':'GOOGL'}, 
                        {'symbol':'META'}, 
                        {'symbol':'BRK-B'}, 
                        {'symbol':'TSLA'}, 
                        ] 
    watch_list = build_analysis_table(custom_watchlist, api_key, vwap_days=105)
else:
    symbol_input = input("Input symbol to research: ").upper()
    one_off_lookup = [{'symbol': symbol_input}]
    watch_list = build_analysis_table(one_off_lookup, api_key, vwap_days=104)

# Create watch list data frame
watch_list_df = pd.DataFrame(watch_list, 
                             columns=['ticker', 'basis', 'price', 
                                      'VWAP', 'GIIB', 'exit', 
                                      'P/E', 'P/B', 'DivYield', 'rating'])

In [10]:
watch_list

[['MSFT', 0, 454.27, 361.81, 131.78, 411.91, 34.97, 10.46, 0.0073, 'Sell'],
 ['AAPL', 0, 211.26, 195.35, 29.71, 224.64, 32.91, 47.28, 0.0049, 'Hold'],
 ['NVDA', 0, 135.4, 105.57, 17.22, 123.76, 45.9, 41.45, 0.0003, 'Sell'],
 ['AMZN', 0, 205.59, 180.33, 73.93, 207.37, 33.43, 7.12, 0, 'Hold'],
 ['GOOG', 0, 167.43, 153.92, 89.69, 176.55, 18.46, 5.81, 0.0051, 'Hold'],
 ['GOOGL', 0, 166.19, 152.19, 89.76, 174.62, 18.32, 5.76, 0.0051, 'Hold'],
 ['META', 0, 640.34, 537.43, 239.98, 620.53, 25.19, 8.7, 0.0033, 'Sell'],
 ['BRK-B', 0, 514.31, 440.78, 591.69, 501.52, 13.72, 1.695, 0, 'Sell'],
 ['TSLA', 0, 349.98, 272.48, 36.01, 322.97, 196.62, 14.79, 0, 'Sell']]

In [11]:
watch_list_df[watch_list_df['rating']=='Buy']

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating


In [12]:
watch_list_df

,ticker,basis,price,VWAP,GIIB,exit,P/E,P/B,DivYield,rating
0,MSFT,0,454.27,361.81,131.78,411.91,34.97,10.460,0.0073,Sell
1,AAPL,0,211.26,195.35,29.71,224.64,32.91,47.280,0.0049,Hold
2,NVDA,0,135.40,105.57,17.22,123.76,45.90,41.450,0.0003,Sell
3,AMZN,0,205.59,180.33,73.93,207.37,33.43,7.120,0.0000,Hold
4,GOOG,0,167.43,153.92,89.69,176.55,18.46,5.810,0.0051,Hold
5,GOOGL,0,166.19,152.19,89.76,174.62,18.32,5.760,0.0051,Hold
6,META,0,640.34,537.43,239.98,620.53,25.19,8.700,0.0033,Sell
7,BRK-B,0,514.31,440.78,591.69,501.52,13.72,1.695,0.0000,Sell
8,TSLA,0,349.98,272.48,36.01,322.97,196.62,14.790,0.0000,Sell


In [13]:
# Machine Learning

In [14]:
# Data processing and clearning
# Must be in numpy array or tf.Dataset object format

In [15]:
# Feature selection and normalization

In [16]:
# Build model

In [17]:
# Train model

In [18]:
# Evaluate model

In [19]:
# Refine model through hyperparameter tuning